In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import string
import pandas as pd
data = pd.read_json("Quora_data2.json")
data.to_csv("Quora_answers.csv")
data.head()

,question,answers
0,How is tea made?,It depends on how you like your tea !\n\nTo ma...
1,How is tea made?,First: There is no such thing as the perfect t...
2,How is tea made?,Ultimately it all depends in preference. Do yo...
3,How is tea made?,How is Tea Made?\n\nWe've broken down the proc...
4,How is tea made?,1. Tea latte\n\nTea lattes are made with steam...


In [2]:
#convert the input text into same casing format
#so that 'text', 'Text' and 'TEXT' are treated the same way
df= data[['answers']]
df["answers_lower"] = df["answers"].str.lower()
df.head()

,answers,answers_lower
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...


In [3]:
#Removal of punctuations
#!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(answers_lower):
    """custom function to remove the punctuation"""
    return answers_lower.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["answers_wo_punct"] = df["answers_lower"].apply(lambda answers_lower: remove_punctuation(answers_lower))
df.head()

,answers,answers_lower,answers_wo_punct
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,it depends on how you like your tea \n\nto mak...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first there is no such thing as the perfect te...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately it all depends in preference do you...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,how is tea made\n\nweve broken down the proces...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 tea latte\n\ntea lattes are made with steame...


In [4]:
#Removal of stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Krina18\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [6]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["answers_wo_stop"] = df["answers_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

,answers,answers_lower,answers_wo_punct,answers_wo_stop
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,it depends on how you like your tea \n\nto mak...,depends like tea make perfect cup tea you’ll n...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first there is no such thing as the perfect te...,first thing perfect tea thing right technique ...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately it all depends in preference do you...,ultimately depends preference like cuppa stron...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,how is tea made\n\nweve broken down the proces...,tea made weve broken process harvest productio...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 tea latte\n\ntea lattes are made with steame...,1 tea latte tea lattes made steamed milk like ...


In [7]:
#Removal of Frequent words
from collections import Counter
cnt = Counter()
for text in df["answers_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('tea', 179),
 ('water', 73),
 ('leaves', 64),
 ('cup', 33),
 ('teas', 31),
 ('time', 27),
 ('green', 25),
 ('milk', 24),
 ('add', 22),
 ('black', 21)]

In [8]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["answers_wo_stopfreq"] = df["answers_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

,answers,answers_lower,answers_wo_punct,answers_wo_stop,answers_wo_stopfreq
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,it depends on how you like your tea \n\nto mak...,depends like tea make perfect cup tea you’ll n...,depends like make perfect you’ll need loose le...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first there is no such thing as the perfect te...,first thing perfect tea thing right technique ...,first thing perfect thing right technique make...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately it all depends in preference do you...,ultimately depends preference like cuppa stron...,ultimately depends preference like cuppa stron...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,how is tea made\n\nweve broken down the proces...,tea made weve broken process harvest productio...,made weve broken process harvest production un...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 tea latte\n\ntea lattes are made with steame...,1 tea latte tea lattes made steamed milk like ...,1 latte lattes made steamed like cafe latte us...


In [9]:
#Removal of rarewords
# Drop the two columns which are no more needed 
df.drop(["answers_wo_punct", "answers_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["answers_wo_stopfreqrare"] = df["answers_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

,answers,answers_lower,answers_wo_stopfreq,answers_wo_stopfreqrare
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,depends like make perfect you’ll need loose le...,depends like make perfect you’ll need loose le...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first thing perfect thing right technique make...,first thing perfect thing right technique make...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately depends preference like cuppa stron...,ultimately depends preference like cuppa stron...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,made weve broken process harvest production un...,made weve broken process harvest production un...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 latte lattes made steamed like cafe latte us...,1 latte lattes made steamed like cafe latte us...


In [10]:
#Stemming
from nltk.stem.porter import PorterStemmer
# Drop the two columns 
#df.drop(["answers_wo_stopfreq", "answers_wo_stopfreqrare"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["answers_stemmed"] = df["answers_lower"].apply(lambda text: stem_words(text))
df.head()

,answers,answers_lower,answers_wo_stopfreq,answers_wo_stopfreqrare,answers_stemmed
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,depends like make perfect you’ll need loose le...,depends like make perfect you’ll need loose le...,it depend on how you like your tea ! to make y...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first thing perfect thing right technique make...,first thing perfect thing right technique make...,first: there is no such thing as the perfect t...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately depends preference like cuppa stron...,ultimately depends preference like cuppa stron...,ultim it all depend in preference. do you like...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,made weve broken process harvest production un...,made weve broken process harvest production un...,how is tea made? we'v broken down the process ...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 latte lattes made steamed like cafe latte us...,1 latte lattes made steamed like cafe latte us...,"1. tea latt tea latt are made with steam milk,..."


In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Krina18\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [20]:
#Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["answers_lemmatized"] = df["answers_stemmed"].apply(lambda text: lemmatize_words(text))
df.head()


,answers,answers_lower,answers_wo_stopfreq,answers_wo_stopfreqrare,answers_stemmed,answers_lemmatized
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,depends like make perfect you’ll need loose le...,depends like make perfect you’ll need loose le...,it depend on how you like your tea ! to make y...,it depend on how you like your tea ! to make y...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first thing perfect thing right technique make...,first thing perfect thing right technique make...,first: there is no such thing as the perfect t...,first: there is no such thing a the perfect te...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately depends preference like cuppa stron...,ultimately depends preference like cuppa stron...,ultim it all depend in preference. do you like...,ultim it all depend in preference. do you like...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,made weve broken process harvest production un...,made weve broken process harvest production un...,how is tea made? we'v broken down the process ...,how is tea made? we'v broken down the process ...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 latte lattes made steamed like cafe latte us...,1 latte lattes made steamed like cafe latte us...,"1. tea latt tea latt are made with steam milk,...","1. tea latt tea latt are made with steam milk,..."


In [22]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Krina18\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [23]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["answers_lemmatized"] = df["answers_stemmed"].apply(lambda text: lemmatize_words(text))
df.head()

,answers,answers_lower,answers_wo_stopfreq,answers_wo_stopfreqrare,answers_stemmed,answers_lemmatized
0,It depends on how you like your tea !\n\nTo ma...,it depends on how you like your tea !\n\nto ma...,depends like make perfect you’ll need loose le...,depends like make perfect you’ll need loose le...,it depend on how you like your tea ! to make y...,it depend on how you like your tea ! to make y...
1,First: There is no such thing as the perfect t...,first: there is no such thing as the perfect t...,first thing perfect thing right technique make...,first thing perfect thing right technique make...,first: there is no such thing as the perfect t...,first: there be no such thing a the perfect te...
2,Ultimately it all depends in preference. Do yo...,ultimately it all depends in preference. do yo...,ultimately depends preference like cuppa stron...,ultimately depends preference like cuppa stron...,ultim it all depend in preference. do you like...,ultim it all depend in preference. do you like...
3,How is Tea Made?\n\nWe've broken down the proc...,how is tea made?\n\nwe've broken down the proc...,made weve broken process harvest production un...,made weve broken process harvest production un...,how is tea made? we'v broken down the process ...,how be tea made? we'v break down the process f...
4,1. Tea latte\n\nTea lattes are made with steam...,1. tea latte\n\ntea lattes are made with steam...,1 latte lattes made steamed like cafe latte us...,1 latte lattes made steamed like cafe latte us...,"1. tea latt tea latt are made with steam milk,...","1. tea latt tea latt be make with steam milk, ..."
